# Analysis for the noun-noun-compounds paper

## Exploratory analysis

We start by loading the data and looking at the points in the original space, before pastes / merges.

In [ ]:
import json
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress, spearmanr
from scipy.optimize import curve_fit
import numpy as np
import sys
sys.path.append('..')
from data import build_dataframe
import os
import data
from util import inv_proportionality

### Old data

In [ ]:
ENTROPIES_FILENAME = '../output/KoplenigEtAl/merged.csv'
SEL_LANGS = ('eng', 'deu', 'nld')
NEW_FILES_DIR = '5_output'
BIBLE_LOCATION = '1_relevant_bibles'
BOOKS = [40, 41, 42, 43, 44, 66]

In [ ]:
df = pd.read_csv(ENTROPIES_FILENAME)

In [ ]:
df_orig_books = df[df['iter_id'] == 0].reset_index(drop=True)

In [ ]:
df_sel_langs = df_orig_books[df_orig_books['bible'].apply(lambda x: any([x.startswith(el) for el in SEL_LANGS]))].reset_index(drop=True)

In [ ]:
df_sel_langs['bible_id'] = df_sel_langs['bible'].apply(lambda x: x.strip().replace('.txt', '').replace('-bible', ''))

In [ ]:
df_sel_langs = df_sel_langs[df_sel_langs['experiment'] == 'pasting'].reset_index(drop=True)

### New data

In [ ]:
new_files = [file for file in os.listdir(NEW_FILES_DIR) if file.startswith('eng-x-bible-') and file.endswith('.txt.csv')]
all_df = []
for i, file in enumerate(new_files):
    df = pd.read_csv(os.path.join(NEW_FILES_DIR, file))
    if len(df) == 0:
        print(f'Skipping {file} because it is empty')
        continue
    df.fillna({'merged_pair': ''}, inplace=True)
    df['filename'] = file
    all_df.append(df)
merged_df = pd.concat(all_df)

There should be one entropy for each book, for each number of merges, and for each filename (the merged_pair is associated 1-1 with n_merges)

In [ ]:
assert all([len(grp) == 3 for _, grp in merged_df.groupby(['book_id', 'n_merges', 'merged_pair', 'filename'])])

In [ ]:
book_ids, n_merges, merged_pair, filename, H_orig, H_order, H_structure, D_order, D_structure = [], [], [], [], [], [], [], [], []
for lbl, grp in merged_df.groupby(['book_id', 'n_merges', 'merged_pair', 'filename']):
    H = {key: grp[grp['text_version'] == key]['entropy'].tolist()[0] for key in grp['text_version'].unique()}
    book_ids.append(lbl[0])
    n_merges.append(lbl[1])
    merged_pair.append(lbl[2])
    filename.append(lbl[3])
    H_orig.append(H['orig'])
    H_order.append(H['shuffled'])
    H_structure.append(H['masked'])
    D_order.append(H['shuffled'] - H['orig'])
    D_structure.append(H['masked'] - H['orig'])
joined_df = pd.DataFrame({'book_id': book_ids, 'n_merges': n_merges, 'merged_pair': merged_pair, 'filename': filename, 
                          'H_orig': H_orig, 'H_order': H_order, 'H_structure': H_structure, 'D_order': D_order, 'D_structure': D_structure})

In [ ]:
assert len(joined_df) == len(merged_df) / 3

In [ ]:
joined_df['filename'] = joined_df['filename'].apply(lambda x: x.strip().replace('.csv', ''))

### Compatibility check

See that the values at 0 merges match between the old and new data

In [ ]:
old_unpasted = df_sel_langs[(df_sel_langs['iter_id'] == 0) & (df_sel_langs['bible_id'].apply(lambda x: x.startswith('eng')))].reset_index(drop=True)

In [ ]:
new_unpasted = joined_df[joined_df['n_merges'] == 0].reset_index(drop=True)

In [ ]:
assert len(old_unpasted) == len(new_unpasted)

In [ ]:
old_unpasted['bible_book'] = old_unpasted.apply(lambda row: f'{row["bible"]}_{row["book_id"]}', 1)
new_unpasted['bible_book'] = new_unpasted.apply(lambda row: f'{row["filename"]}_{row["book_id"]}', 1)

In [ ]:
assert len(old_unpasted) == old_unpasted['bible_book'].nunique()
assert len(new_unpasted) == new_unpasted['bible_book'].nunique()

In [ ]:
old_and_new = old_unpasted.merge(new_unpasted, on='bible_book', how='inner', validate='1:1')

In [ ]:
assert len(old_and_new) == len(old_unpasted)

In [ ]:
key_map = {'orig': 'H_orig', 'shuffled': 'H_order', 'masked': 'H_structure'}
for key, val in key_map.items():
    old_and_new[f'{val}_diff'] = old_and_new.apply(lambda row: abs(row[key] - row[val]), 1)
    old_and_new[f'{val}_mean'] = old_and_new.apply(lambda row: 0.5 * (row[key] + row[val]), 1)
    old_and_new[f'{val}_fracdiff'] = old_and_new.apply(lambda row: row[f'{val}_diff'] / row[f'{val}_mean'], 1)
all_frac_diffs = []
for val in key_map.values():
    all_frac_diffs += old_and_new[f'{val}_fracdiff'].tolist()
assert len(all_frac_diffs) == 3 * len(old_and_new)

In [ ]:
plt.hist(all_frac_diffs)

In [ ]:
largest_percentual_difference = max(all_frac_diffs)*100
print(f'The largest difference between old and new results is {largest_percentual_difference:.2f}%')
assert largest_percentual_difference < 5

## Exclude old texts?

Plot the starting quantities (no pastes) to see if we need to exclude certain bibles, especially those in old variants of a language. Look at this also by looking at the metadata in the files.

In [ ]:
lang_color = {'eng': 'b', 'nld': 'r', 'deu': 'g'}

In [ ]:
for book_name in df_sel_langs['book'].unique():
    book_df = df_sel_langs[df_sel_langs['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    x = book_df['D_order'].tolist()
    y = book_df['D_structure'].tolist()
    point_color = book_df['bible'].apply(lambda x: lang_color[x[:3]]).tolist()
    point_legend = book_df['bible'].apply(lambda x: x[:3]).tolist()
    labels = book_df['bible_id'].tolist()
    fig, ax = plt.subplots()
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        labels = lang_df['bible_id'].apply(lambda x: x[6:]).tolist()
        ax.scatter(x, y, c=point_color, label=lang)
        for i, txt in enumerate(labels):
            ax.annotate(txt, (x[i], y[i]), rotation=45)
    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

I do not see any outliers there. The next step is to see if the bible dates are old. For this we need to read the file metadata.

In [ ]:
bible_data = []
for bible_filename in df_sel_langs['bible'].unique():
    with open(os.path.join(BIBLE_LOCATION, bible_filename)) as f:
        lines = f.readlines()
    comments, _, _ = data.split_pbc_bible_lines(lines, False)
    comments['filename'] = bible_filename
    bible_data.append(comments)
df = pd.DataFrame(bible_data)

In [ ]:
df[['filename', 'language_name', 'year_short', 'year_long']].sort_values('year_short').head(10)

We see three potentially interesting scenarios:
1. Bibles without a year
2. Old German bibles
3. Old Dutch bibles
4. Old English bibles

### Bibles without a year

deu-x-bible-bolsinger.txt is supposedly a 1545 Luther bible. However, the curators of the corpus wrote ``It is clearly NOT luther's 1545 version. The language is contemporary German, maybe a personal version of mr. Bolsinger?'' Do, we will exclude this bible conservatively.

eng-x-bible-treeoflife.txt was developed by a society founded in 2009, so it must be in Modern English, and we will keep it in the corpus.

deu-x-bible-erben.txt was translated by Johann Albrecht Bengel (born 1687) sometime before [1739](https://de.wikipedia.org/wiki/Johann_Albrecht_Bengel#Bengel_als_Textkritiker). So we can write the approximate year in the file and re-categorize it.

In [ ]:
excluded_bibles = ['deu-x-bible-bolsinger.txt']

In [ ]:
df.loc[df['filename'] == 'eng-x-bible-treeoflife.txt', ['year_short', 'year_long']] = '2009-2025'
df.loc[df['filename'] == 'deu-x-bible-erben.txt', ['year_short','year_long']] = '1687-1739'

In [ ]:
df[df['filename'].apply(lambda x: x not in excluded_bibles)][['filename', 'language_name', 'year_short', 'year_long']].sort_values('year_short').head(10)

### Old German bibles

In [ ]:
df[(df['filename'].apply(lambda x: x not in excluded_bibles)) & (df['filename'].apply(lambda x: x.startswith('deu')))][['filename', 'year_short', 'year_long']].sort_values('year_short')

German came to its modern form [in the 19th century](https://en.wikipedia.org/wiki/History_of_German). So, we should exclude the bibles from before that period.

In [ ]:
excluded_bibles += ['deu-x-bible-luther1545letztehand.txt', 'deu-x-bible-erben.txt']

### Old Dutch bibles

In [ ]:
df[(df['filename'].apply(lambda x: x not in excluded_bibles)) & (df['filename'].apply(lambda x: x.startswith('nld')))][['filename', 'year_short', 'year_long']].sort_values('year_short')

Dutch evolved little since the 17th century. Still, we will put the cut in 1800, which is the same as German, because it was the time when all regions east of the Netherlands standardized their version of German. Also, because I've looked at parts of the 1637 text, and it doesn't look just like contemporary Dutch.

In [ ]:
excluded_bibles += ['nld-x-bible-statenvertaling.txt']

### Old English bibles

In [ ]:
df[(df['filename'].apply(lambda x: x not in excluded_bibles)) & (df['filename'].apply(lambda x: x.startswith('eng')))][['filename', 'year_short', 'year_long']].sort_values('year_short')

Although English from the 17th century is considered Early Modern English, I will place a cut at 1800 as well to use the same date for all three languages, and because the language of Shakespeare has enough differences with the modern language in terms of spelling that I'd like to avoid using it.

In [ ]:
excluded_bibles += ['eng-x-bible-kingjames.txt']

Now I need to remove the excluded_bibles from the datasets and plot the Koplenig et al plots from above once again.

In [ ]:
old_data = df_sel_langs[df_sel_langs['bible'].apply(lambda x: x not in excluded_bibles)].reset_index(drop=True)
assert len(old_data) < len(df_sel_langs)

In [ ]:
new_data = joined_df[joined_df['filename'].apply(lambda x: x not in excluded_bibles)].reset_index(drop=True)
assert len(new_data) < len(joined_df)

In [ ]:
assert len(old_data[old_data.apply(lambda row: row['iter_id'] != 0 or row['experiment'] != 'pasting', 1)]) == 0

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    x = book_df['D_order'].tolist()
    y = book_df['D_structure'].tolist()
    point_color = book_df['bible'].apply(lambda x: lang_color[x[:3]]).tolist()
    point_legend = book_df['bible'].apply(lambda x: x[:3]).tolist()
    labels = book_df['bible_id'].tolist()
    fig, ax = plt.subplots()
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        labels = lang_df['bible_id'].apply(lambda x: x[6:]).tolist()
        ax.scatter(x, y, c=point_color, label=lang)
        for i, txt in enumerate(labels):
            ax.annotate(txt, (x[i], y[i]), rotation=45)
    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

## Averaging values

We are interested in knowing the word order information and word structure information contained in a given language per unit character *on average*. In a way, the different estimates we have of the word-order and word-structure information are independent estimates of the same quantity. Therefore, we could compute standard statistics on them and then combine them in a plot.

I'm not entirely sure how the statistics should be combined, so I will study this issue briefly first.

In [ ]:
old_data[(old_data['bible'].apply(lambda x: x.startswith('eng'))) & (old_data['book_id'] == 40)][['orig', 'shuffled', 'masked', 'D_order', 'D_structure']].apply(['mean', 'std'])

In [ ]:
A = 2.117682
B = 1.580128
C = 1.512396
sigma_A = 0.037449
sigma_B = 0.076276
sigma_C = 0.065962
print('D_order', A-C, 0.605286)
print('D_structure', B-C, 0.067732)

In [ ]:
sigma_x = np.sqrt(sigma_A**2+sigma_C**2)

In [ ]:
sigma_y = np.sqrt(sigma_B**2+sigma_C**2)

In [ ]:
print('sigma_order', sigma_x, 0.049302)

In [ ]:
print('sigma_structure', sigma_y, 0.011786)

I think we overestimate the standard deviation if we do error propagation because it is meant for cases with uncorrelated errors. So we will take the standard deviation of the difference.

In [ ]:
old_data.head()

In [ ]:
old_data['language'] = old_data['bible'].apply(lambda x: x[:3])

In [ ]:
old_data[['book', 'language', 'D_order', 'D_structure']].groupby(['book', 'language']).agg(['mean', 'std'])

Figure out how to collapse this back into a simple dataframe, then put it on the plot like Koplenig et al.

Actually, that is not necessary. We can make the calculation at the time we make the scatter plot.

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        ax.errorbar(x=mean_x, y=mean_y, xerr=error_x, yerr=error_y, c=point_color, label=lang)
    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

With these plots it would seem that languages are indistinguishable. This counters the intuition that we get from looking at the plot. What if we take all estimates and compute sample statistics on those?

In [ ]:
del book_df
fig, ax = plt.subplots()
for lang, point_color in lang_color.items():
    lang_df = old_data[old_data['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
    x = lang_df['D_order'].tolist()
    y = lang_df['D_structure'].tolist()
    mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
    # 95% confidence interval should be at 2std
    error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
    ax.errorbar(x=mean_x, y=mean_y, xerr=error_x, yerr=error_y, c=point_color, label=lang)
ax.legend()
plt.xlabel('Word order information')
plt.ylabel('Word structure information')
plt.title(book_name)

These look more indistinguishable. I think we'd have to consult an expert statistician to see how they think this should be done. For sure I think the last plot does not make sense, because the right way to combine those would have been to compute the entropy on the entire sample rather than on the individual books.

Also, since we're comparing against Koplenig et al, and they averaged across translations (see "Materials and methods"), we can also follow their methodology and average. The question is whether to average for a given number of pastes, or to look at the paste history of a single translation.

## Koplenig-et-al-style plots with average values

We now add the best-fit lines produced by Koplenig et al

In [ ]:
fit_params = pd.read_csv('9_koplenig_et_al_fit_params.csv', sep=';')

In [ ]:
fit_params['beta_0'] = fit_params['beta_0'].apply(lambda x: float(x.replace(',', '.')))
fit_params['beta_1'] = fit_params['beta_1'].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
fit_params

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    # Plot the fit line from Koplenig et al
    fit_x = np.arange(book_df['D_order'].min(), book_df['D_order'].max(), (book_df['D_order'].max() - book_df['D_order'].min()) / 100)
    beta_0 = fit_params[fit_params['book'] == book_name]['beta_0'].tolist()[0]
    beta_1 = fit_params[fit_params['book'] == book_name]['beta_1'].tolist()[0]
    fit_y = [beta_0 + beta_1 / el for el in fit_x]
    ax.plot(fit_x, fit_y, linestyle='dashed')
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        ax.scatter(x=mean_x, y=mean_y, c=point_color, label=lang)
    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

We could instead fit our own lines.

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    x_data, y_data = [], []
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        x_data.append(np.mean(x))
        y_data.append(np.mean(y))
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        ax.scatter(x=mean_x, y=mean_y, c=point_color, label=lang)
    popt, pcov = curve_fit(inv_proportionality, x_data, y_data, p0=(1, 1))
    a, b = popt
    # Create a range for x values to plot the curve
    x_range = np.linspace(min(x_data)*0.9, max(x_data)*1.1, 400)
    
    # Calculate the corresponding y values using the best-fit parameters
    y_range = inv_proportionality(x_range, a, b)
    
    # Plot the data and the best-fit curve
    plt.plot(x_range, y_range, 'black', label='Fit: y = %.2fx^-1 + %.2f' % (a, b), linestyle='dashed')
    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

## Final analysis

We now have all the tools ready, and it is time to look at the data I generated. I will follow these steps:
1. for each book and for each number of merges, average D_order and D_structure across translations
2. this will give me an ordered sequence of D_order, D_structure
3. On each plot like the ones above, I can add these new points, along with their labels (number of pastes)

First, let's do that with the old pastes. What effect do they have?

In [ ]:
full_old_data_df = pd.read_csv(ENTROPIES_FILENAME)

In [ ]:
old_eng_pastes = full_old_data_df[full_old_data_df.apply(lambda row: row['bible'].startswith('eng') and row['experiment'] == 'pasting' and row['bible'] not in excluded_bibles, 1)]

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    x_data, y_data = [], []
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        x_data.append(np.mean(x))
        y_data.append(np.mean(y))
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        # Add diagonal labels for each datapoint
        ax.scatter(x=mean_x, y=mean_y, c=point_color, label=lang)
        ax.annotate(lang, (mean_x[0], mean_y[0]), rotation=45)
    popt, pcov = curve_fit(inv_proportionality, x_data, y_data, p0=(1, 1))
    a, b = popt
    # Create a range for x values to plot the curve
    x_range = np.linspace(min(x_data)*0.9, max(x_data)*1.1, 400)
    
    # Calculate the corresponding y values using the best-fit parameters
    y_range = inv_proportionality(x_range, a, b)
    
    # Plot the fit line from Koplenig et al
    fit_x = np.arange(book_df['D_order'].min(), book_df['D_order'].max(), (book_df['D_order'].max() - book_df['D_order'].min()) / 100)
    beta_0 = fit_params[fit_params['book'] == book_name]['beta_0'].tolist()[0]
    beta_1 = fit_params[fit_params['book'] == book_name]['beta_1'].tolist()[0]
    fit_y = [beta_0 + beta_1 / el for el in fit_x]
    ax.plot(fit_x, fit_y, linestyle='dashed', label='Koplenig et al fit line')

    # Plot the old pastes data
    n_merge_quantities = old_eng_pastes[(old_eng_pastes['book'] == book_name) & (old_eng_pastes['iter_id']<=200) & (old_eng_pastes['iter_id']>0)][['iter_id', 'D_order', 'D_structure']].groupby('iter_id').mean().reset_index(drop=False)
    x = n_merge_quantities['D_order'].tolist()
    y = n_merge_quantities['D_structure'].tolist()
    ax.scatter(x, y, label='any word pair pastes') # , c=point_color, label=lang)
    labels = n_merge_quantities['iter_id'].tolist()
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x[i], y[i]), rotation=45)

    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)
    plt.savefig(f'10_figs/all_pastes_{book_name}.png')

The good news is that even at just 100 merges we see the movement in the right direction that we were expecting. Let's see if we can observe it with the new data.

In [ ]:
for lbl, grp in new_data.groupby(['book_id', 'n_merges']):
    assert len(grp) == grp['filename'].nunique()

In [ ]:
book_id_map = old_data[['book_id', 'book']].drop_duplicates()

In [ ]:
new_data_book = new_data.merge(book_id_map, on='book_id', how='left')

In [ ]:
assert len(new_data_book) == len(new_data)

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    x_data, y_data = [], []
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        x_data.append(np.mean(x))
        y_data.append(np.mean(y))
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        ax.scatter(x=mean_x, y=mean_y, c=point_color, label=lang)
        ax.annotate(lang, (mean_x[0], mean_y[0]), rotation=45)
    popt, pcov = curve_fit(inv_proportionality, x_data, y_data, p0=(1, 1))
    a, b = popt
    # Create a range for x values to plot the curve
    x_range = np.linspace(min(x_data)*0.9, max(x_data)*1.1, 400)
    
    # Calculate the corresponding y values using the best-fit parameters
    y_range = inv_proportionality(x_range, a, b)
    
    # Plot the old pastes data
    n_merge_quantities = new_data_book[new_data_book['book'] == book_name][['n_merges', 'D_order', 'D_structure']].groupby('n_merges').mean().reset_index(drop=False)
    n_merge_quantities = n_merge_quantities[n_merge_quantities['n_merges'].apply(lambda x: x % 10 == 0)].reset_index()
    x = n_merge_quantities['D_order'].tolist()
    y = n_merge_quantities['D_structure'].tolist()
    ax.scatter(x, y) # , c=point_color, label=lang)
    labels = n_merge_quantities['n_merges'].tolist()
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x[i], y[i]), rotation=45)

    # Plot the fit line from Koplenig et al
    fit_x = np.arange(book_df['D_order'].min(), book_df['D_order'].max(), (book_df['D_order'].max() - book_df['D_order'].min()) / 100)
    beta_0 = fit_params[fit_params['book'] == book_name]['beta_0'].tolist()[0]
    beta_1 = fit_params[fit_params['book'] == book_name]['beta_1'].tolist()[0]
    fit_y = [beta_0 + beta_1 / el for el in fit_x]
    ax.plot(fit_x, fit_y, linestyle='dashed', label='Koplenig et al fit line')

    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)
    plt.savefig(f'10_figs/nn_pastes_{book_name}.png')

### What if we only include n_merges that are available across all translations?

In [ ]:
for book, filename_n_merges in new_data_book[['filename', 'book', 'n_merges']].groupby('book'):
    print(book)
    max_n_merges = [n_merges['n_merges'].max() for _, n_merges in filename_n_merges.groupby('filename')]
    min_max_n_merges = min(max_n_merges)
    print(book, min_max_n_merges)

In [ ]:
for book, filename_n_merges in new_data_book[['filename', 'book', 'n_merges']].groupby('book'):
    print(book)
    max_n_merges = [n_merges['n_merges'].max() for _, n_merges in filename_n_merges.groupby('filename')]
    plt.hist(max_n_merges)
    plt.show()

To understand this better, I need to know how many verses from each book are included in each of the English bibles.

In [ ]:
files, books, verses = [], [], []
for file in os.listdir(BIBLE_LOCATION):
    if not file.startswith('eng'):
        continue
    if file in excluded_bibles:
        print('skipping excluded', file)
        continue
    bible = data.parse_file(os.path.join(BIBLE_LOCATION, file), 'pbc').join_by_toc()
    for book_id in BOOKS:
        book = book_id_map[book_id_map['book_id'] == book_id]['book'].tolist()[0]
        n_verses = len(bible[2][book_id])
        files.append(file)
        books.append(book)
        verses.append(n_verses)
verse_df = pd.DataFrame({'file': files, 'book': books, 'n_verses': verses})

In [ ]:
for lbl, grp in verse_df.groupby('book'):
    print('---------')
    print(lbl)
    print('---------')
    print(grp.sort_values('n_verses'))

Let's exclude all bibles that contain fewer than 90% of the maximum number of verses for at least one book.

In [ ]:
excluded_bibles += ['eng-x-bible-books.txt', 'eng-x-bible-contemporary.txt', 'eng-x-bible-interconfessional.txt', 'eng-x-bible-scriptures.txt', 'eng-x-bible-standard.txt']

In [ ]:
print(len(new_data_book))

In [ ]:
new_data_long = new_data_book[new_data_book['filename'].apply(lambda x: x not in excluded_bibles)].reset_index(drop=True)

In [ ]:
print(len(new_data_long))

In [ ]:
for book_name in old_data['book'].unique():
    book_df = old_data[old_data['book'] == book_name].reset_index(drop=True)
    assert len(book_df) == book_df['bible_id'].nunique(), (book_name, str(len(book_df)), str(book_df['bible_id'].nunique()))
    fig, ax = plt.subplots()
    x_data, y_data = [], []
    for lang, point_color in lang_color.items():
        lang_df = book_df[book_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
        x = lang_df['D_order'].tolist()
        y = lang_df['D_structure'].tolist()
        x_data.append(np.mean(x))
        y_data.append(np.mean(y))
        mean_x, mean_y = [[np.mean(el)] for el in (x, y)]
        # 95% confidence interval should be at 2std
        error_x, error_y = [[np.std(el) * 2] for el in (x, y)]
        ax.scatter(x=mean_x, y=mean_y, c=point_color, label=lang)
    popt, pcov = curve_fit(inv_proportionality, x_data, y_data, p0=(1, 1))
    a, b = popt
    # Create a range for x values to plot the curve
    x_range = np.linspace(min(x_data)*0.9, max(x_data)*1.1, 400)
    
    # Calculate the corresponding y values using the best-fit parameters
    y_range = inv_proportionality(x_range, a, b)
    
    # Plot the best-fit curve
    plt.plot(x_range, y_range, 'black', label='Fit: y = %.2fx^-1 + %.2f' % (a, b), linestyle='dashed')

    # Plot the old pastes data
    n_merge_quantities = new_data_long[new_data_long['book'] == book_name][['n_merges', 'D_order', 'D_structure']].groupby('n_merges').mean().reset_index(drop=False)
    n_merge_quantities = n_merge_quantities[n_merge_quantities['n_merges'].apply(lambda x: x % 10 == 0)].reset_index()
    x = n_merge_quantities['D_order'].tolist()
    y = n_merge_quantities['D_structure'].tolist()
    ax.scatter(x, y) # , c=point_color, label=lang)
    labels = n_merge_quantities['n_merges'].tolist()
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x[i], y[i]), rotation=45)

    ax.legend()
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(book_name)

# Trend for each individual bible

In [ ]:
book_name = 'Mark'
old_df = old_data[old_data['book'] == book_name].reset_index()
new_df = new_data_long[(new_data_long['book'] == book_name) & (new_data_long['n_merges'].apply(lambda x: x % 5 == 0))].reset_index()
lang_central_data = {}
for lang in lang_color.keys():
    lang_df = old_df[old_df['bible'].apply(lambda x: x.startswith(lang))].reset_index(drop=True)
    x = lang_df['D_order'].tolist()
    y = lang_df['D_structure'].tolist()
    lang_central_data[lang] = [[np.mean(el)] for el in (x, y)]
for filename in new_data_long['filename'].unique():
    file_df = new_df[new_df['filename'] == filename].reset_index()
    assert len(file_df) == file_df['n_merges'].nunique()
    x, y, labels = [file_df[col].tolist() for col in ('D_order', 'D_structure', 'n_merges')]
    fig, ax = plt.subplots()
    for lang, central_data in lang_central_data.items():
        ax.scatter(central_data[0], central_data[1])
        ax.annotate(lang, (central_data[0][0], central_data[1][0]), rotation=45)
    ax.scatter(x, y)
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x[i], y[i]), rotation=45)
    plt.xlabel('Word order information')
    plt.ylabel('Word structure information')
    plt.title(filename)

# Try to estimate the confidence interval of the Koplenig et al. best-fit lines

In [ ]:
class FitResults:
    def __init__(self, x_range, y_range, y_lower, y_upper, a, b):
        self.x_range = x_range
        self.y_range = y_range
        self.y_lower = y_lower
        self.y_upper = y_upper
        self.a = a
        self.b = b


def fit_data(x_data: list, y_data: list) -> FitResults:
    # noinspection PyTupleAssignmentBalance
    popt, pcov = curve_fit(inv_proportionality, x_data, y_data, p0=(1, 1))
    a, b = popt
    # Create a range for x values to plot the curve
    x_range = np.linspace(min(x_data) * 0.9, max(x_data) * 1.1, 400)

    # Calculate the corresponding y values using the best-fit parameters
    y_range = inv_proportionality(x_range, a, b)

    # Calculate confidence intervals
    perr = np.sqrt(np.diag(pcov))
    a_err, b_err = perr

    # Calculate 95% (thus multiplied by 2) confidence intervals for the fit
    y_upper = inv_proportionality(x_range, a + 2 * a_err, b + 2 * b_err)
    y_lower = inv_proportionality(x_range, a - 2 * a_err, b - 2 * b_err)

    return FitResults(x_range, y_range, y_lower, y_upper, a, b)

In [ ]:
# Create best-fit lines like in Koplenig et al., but with confidence intervals
df = pd.read_csv(ENTROPIES_FILENAME)
df['language'] = df['bible'].apply(lambda x: x[:3])
excluded_languages = {'mya'}
df = df[df['language'].apply(lambda x: x not in excluded_languages)].reset_index(drop=True)
print(df.columns)
central_points = df[df.apply(lambda r: r['iter_id'] == 0 and r['experiment'] == 'pasting', 1)].reset_index(drop=True)
label_langs = {'chr', 'cmn', 'deu', 'eng', 'esk', 'grc', 'mya', 'tam', 'qvw', 'vie', 'xuo', 'zul'}
for book, grp in central_points.groupby('book'):
    assert len(grp) == grp['bible'].nunique()
    # Average values from the same language, as in Koplenig et al.
    lang_vals = grp[['language', 'D_order', 'D_structure']].groupby('language').mean().reset_index()
    # Excluded non-physical values
    lang_vals = lang_vals[lang_vals.apply(lambda r: r['D_order'] >= 0 and r['D_structure'] >= 0, 1)].reset_index(
        drop=True
    )
    # Select the languages that will receive a special marker
    label_df = lang_vals[lang_vals['language'].apply(lambda x: x in label_langs)]
    no_label_df = lang_vals[lang_vals['language'].apply(lambda x: x not in label_langs)]
    assert len(label_df) > 0 and len(no_label_df) > 0 and len(label_df) + len(no_label_df) == len(lang_vals)
    # Fit a line through the data points
    fit_res = fit_data(lang_vals['D_order'].tolist(), lang_vals['D_structure'].tolist())
    # Now plot the data points and the fit line
    fig, ax = plt.subplots()
    # Points without a label
    ax.scatter(no_label_df['D_order'].tolist(), no_label_df['D_structure'].tolist())
    # Points with a label
    x_label = label_df['D_order'].tolist()
    y_label = label_df['D_structure'].tolist()
    ax.scatter(x_label, y_label, c='orange')
    labels = label_df['language'].tolist()
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x_label[i], y_label[i]), rotation=45)
    # Fit line
    plt.plot(fit_res.x_range, fit_res.y_range, '-', label=f'{fit_res.b:.2f} + {fit_res.a:.2f} / x')
    plt.fill_between(fit_res.x_range, fit_res.y_lower, fit_res.y_upper, alpha=0.2, label='95% CI')
    plt.xlabel('D_order')
    plt.ylabel('D_structure')
    plt.legend()
    plt.title(f'{book} (N={len(lang_vals)})')
    plt.show()